In [1]:
import numpy as np
from CrossSections import Transfer_SigmaV, Transfer_sigma
from scipy.optimize import minimize


## Fits con Datos de Camila

In [2]:
#Import Data from Camila Correa
CamilaData_Fig6 = np.loadtxt('Data-Sets/Data_Fig6_Correa_2021.txt')

VelocityData = CamilaData_Fig6[:,0]
Velocity_16_Percentile = CamilaData_Fig6[:,1]
Velocity_84_Percentile = CamilaData_Fig6[:,2]
VelocityData_Err = ( (VelocityData - Velocity_16_Percentile) + (Velocity_84_Percentile - VelocityData) ) /2

CrossSectionData = CamilaData_Fig6[:,3]
CrossSectionData_16_Percentile = CamilaData_Fig6[:,4]
CrossSectionData_84_Percentile = CamilaData_Fig6[:,5]

CrossSectionData_Err = ((CrossSectionData - CrossSectionData_16_Percentile) + (CrossSectionData_84_Percentile - CrossSectionData) )/2


In [10]:
def CamilaTranser(v, M, m):
    alpha = 0.3
    w = 300*(M/10)*(10/m)
    sigma0T = 274.4*(alpha/0.01)**2*(m/10)*(10/M)**4
    return sigma0T*4*w**4/v**4 * (2*np.log(1 + v**2/(2*w**2)) - np.log(1 + v**2/w**2) )


# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=VelocityData, data=CrossSectionData, err=(VelocityData_Err, CrossSectionData_Err)):
    #Compute model
    M, m = free_params
    model = [CamilaTranser(x, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y# + chi2x


Fits for Figure 6

In [11]:

M_ini, m_ini = 50, 6
initial = [M_ini, m_ini]

soln = minimize(compute_chi2, initial, method='Nelder-Mead')

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} MeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf) )/(len(VelocityData) - 1) )


Best Fit Parameters
M = 69.28 MeV
m = 3.44 GeV
Chi-squared: 3.979851002097949


In [12]:
compute_chi2([53.933, 6.605])/(len(VelocityData) - 1) 

410.09009054320126

Fits for Figure 7:

In [ ]:
CamilaData_Fig7 = np.loadtxt('Data-Sets/Data_Fig7_Correa_2021.txt')
VelocityData = CamilaData_Fig7[:,0]
Velocity_16_Percentile = CamilaData_Fig7[:,1]
Velocity_84_Percentile = CamilaData_Fig7[:,2]
VelocityData_Err = ( (VelocityData - Velocity_16_Percentile) + (Velocity_84_Percentile - VelocityData) ) /2

CrossSectionData = CamilaData_Fig7[:,3]
CrossSectionData_16_Percentile = CamilaData_Fig7[:,4]
CrossSectionData_84_Percentile = CamilaData_Fig7[:,5]

CrossSectionData_Err = ((CrossSectionData - CrossSectionData_16_Percentile) + (CrossSectionData_84_Percentile - CrossSectionData) )/2


In [ ]:

def Integrand_Camila_SigmaV(v, v0, M, m):
    return CamilaTranser(v, M, m)*v*np.exp(-0.5*v**2/v0**2)*v**2

from scipy.integrate import quad

def Camila_SigmaV(v0, M, m):
    sigma2_MB = v0**2*np.pi*(3*np.pi - 8)/np.pi
    vmax = 2*np.sqrt(sigma2_MB)

    Prefactor = 4*np.pi/((2*np.pi*v0**2)**1.5 * m)
    Integral = quad(Integrand_Camila_SigmaV, 0.1, vmax, args=(v0, M, m))[0]
    return Prefactor*Integral

# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=VelocityData, data=CrossSectionData, err=(VelocityData_Err, CrossSectionData_Err)):
    #Compute model
    M, m = free_params
    model = [Camila_SigmaV(x, M, m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y #+ chi2x

In [ ]:
soln = minimize(compute_chi2, initial, method='Nelder-Mead')

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} MeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))

## Fits con datos de Yu

In [ ]:
# For the fit
OrderedData = np.loadtxt('Data-Sets/Datos_ordenados.csv')
x_data = OrderedData[:,0]
y_data = OrderedData[:,1]
err_x_data = OrderedData[:,2]
err_y_data = OrderedData[:,3]


In [ ]:
def NormalTranser(v, M, m):
    alpha = 1/137
    w = 300*(M/10)*(10/m)
    sigma0T = 274.4*(alpha/0.01)**2*(m/10)*(10/M)**4
    return sigma0T*4*w**4/v**4 * (np.log(1 + v**2/(w**2)) - (v/w)**2/(1 + (v/w)**2) )

def Integrand_Normal_SigmaV(v, v0, M, m):
    return NormalTranser(v, M, m)*v*np.exp(-0.5*v**2/v0**2)*v**2

from scipy.integrate import quad

def Normal_SigmaV(v0, M, m):
    sigma2_MB = v0**2*np.pi*(3*np.pi - 8)/np.pi
    vmax = 2*np.sqrt(sigma2_MB)

    Prefactor = 4*np.pi/((2*np.pi*v0**2)**1.5 * m)
    Integral = quad(Integrand_Normal_SigmaV, 0.0, vmax, args=(v0, M, m))[0]
    return Prefactor*Integral

In [ ]:

M_ini, m_ini = 10, 10
initial = [M_ini, m_ini]
bnds= [(0.01, 1000), (0.01, 1000)]

# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=x_data, data=y_data, err=(err_x_data, err_y_data)):
    #Compute model
    M, m = free_params
    model = [Normal_SigmaV(x, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y + chi2x

soln = minimize(compute_chi2, initial, method='Nelder-Mead', bounds=bnds)

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} MeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))


Standard Cross Section but with Camila Data

In [ ]:
def compute_chi2(free_params, x=VelocityData, data=CrossSectionData, err=(VelocityData_Err, CrossSectionData_Err)):
    #Compute model
    M, m = free_params
    model = [Normal_SigmaV(x, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y #+ chi2x


M_ini, m_ini = 1, 10
initial = [M_ini, m_ini]


soln = minimize(compute_chi2, initial, method='Nelder-Mead')

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} MeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))

## Particle Physics Cross Section with Camila Data

Data From Dwarfs

In [ ]:
#Import Data from Camila Correa
CamilaData_Fig6 = np.loadtxt('Data-Sets/Data_Fig6_Correa_2021.txt')

VelocityData = CamilaData_Fig6[:,0]
Velocity_16_Percentile = CamilaData_Fig6[:,1]
Velocity_84_Percentile = CamilaData_Fig6[:,2]
VelocityData_Err = ( (VelocityData - Velocity_16_Percentile) + (Velocity_84_Percentile - VelocityData) ) /2

CrossSectionData = CamilaData_Fig6[:,3]
CrossSectionData_16_Percentile = CamilaData_Fig6[:,4]
CrossSectionData_84_Percentile = CamilaData_Fig6[:,5]

CrossSectionData_Err = ((CrossSectionData - CrossSectionData_16_Percentile) + (CrossSectionData_84_Percentile - CrossSectionData) )/2

In [ ]:
# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=VelocityData, data=CrossSectionData, err=(VelocityData_Err, CrossSectionData_Err)):
    #Compute model
    M, m = free_params
    model = [Transfer_sigma(x, g=0.3, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y #+ chi2x


M_ini, m_ini = 10, 10
initial = [M_ini, m_ini]
bnds= [(1, 1000), (.01, 1000)]


soln = minimize(compute_chi2, initial, method='Nelder-Mead', bounds=bnds)

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} MeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))

Data From clusters

In [ ]:
CamilaData_Fig7 = np.loadtxt('Data-Sets/Data_Fig7_Correa_2021.txt')
VelocityData = CamilaData_Fig7[:,0]
Velocity_16_Percentile = CamilaData_Fig7[:,1]
Velocity_84_Percentile = CamilaData_Fig7[:,2]
VelocityData_Err = ( (VelocityData - Velocity_16_Percentile) + (Velocity_84_Percentile - VelocityData) ) /2

CrossSectionData = CamilaData_Fig7[:,3]
CrossSectionData_16_Percentile = CamilaData_Fig7[:,4]
CrossSectionData_84_Percentile = CamilaData_Fig7[:,5]

CrossSectionData_Err = ((CrossSectionData - CrossSectionData_16_Percentile) + (CrossSectionData_84_Percentile - CrossSectionData) )/2


In [ ]:
# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=VelocityData, data=CrossSectionData, err=(VelocityData_Err, CrossSectionData_Err)):
    #Compute model
    M, m = free_params
    model = [Transfer_SigmaV(x, g=0.3, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y #+ chi2x


M_ini, m_ini = 0.01, 100
initial = [M_ini, m_ini]
bnds= [(.001, 100), (.01, 1000)]


soln = minimize(compute_chi2, initial, method='Nelder-Mead', bounds=bnds)

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} GeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))

Data from Yu

In [ ]:
# For the fit
OrderedData = np.loadtxt('Data-Sets/Datos_ordenados.csv')
x_data = OrderedData[:,0]
y_data = OrderedData[:,1]
err_x_data = OrderedData[:,2]
err_y_data = OrderedData[:,3]

In [ ]:

M_ini, m_ini = .01, 10
initial = [M_ini, m_ini]
bnds= [(0.001, 100), (0.01, 1000)]

# Define a chi square distribution to use as input in emcee
def compute_chi2(free_params, x=x_data, data=y_data, err=(err_x_data, err_y_data)):
    #Compute model
    M, m = free_params
    model = [Transfer_SigmaV(x, M=M, m=m) for x in x]

    errx, erry = err
    #chi2 computation
    chi2y= np.sum((data-model)**2/erry**2)
    chi2x= np.sum((x - 2*x*np.sqrt(2/np.pi))**2/errx**2)
    return chi2y + chi2x

soln = minimize(compute_chi2, initial, method='Nelder-Mead', bounds=bnds)

M_bf, m_bf = soln.x

print("Best Fit Parameters")
print('M = {0:.2f} GeV'.format(M_bf))
print('m = {0:.2f} GeV'.format(m_bf))
print("Chi-squared:", compute_chi2((M_bf, m_bf)))